In [ ]:
# Need help identifying the environment from your current notebook?
import sys
sys.executable
!which python3
!which pip
!pip -V

In [ ]:
# pip install google-adk

In [ ]:
# !pip install python-dotenv

In [ ]:
# !pip install yfinance --upgrade
# !python3 -m pip install yfinance --upgrade

In [ ]:
# !pip install feedparser

In [ ]:
from dotenv import load_dotenv
import os

try:
    load_dotenv()
    api_key = os.getenv("GOOGLE_API_KEY")
    if api_key is None:
        raise Exception("GOOGLE_API_KEY is missing")
    print("✅ Gemini API key setup complete.")
    # print(api_key)
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")


In [ ]:
from google.adk.agents import Agent, SequentialAgent, LlmAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types
from google.adk.models.google_llm import Gemini

print("✅ ADK components imported successfully.")

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [ ]:
# First ADK Agent
root_agent = Agent(name="helpful_assistant", model="gemini-2.5-flash-lite", 
                   description="a simple assistant that can answer questions",
                   instruction="you are a helpful assistant. Use Google search for current info or if unsure.",
                   tools=[google_search])
print("Root Agent defined.")

In [ ]:
# Run your agent
runner = InMemoryRunner(agent=root_agent)
print("runner created.")

In [ ]:
# run in debug mode for prototyping
response = await runner.run_debug("What is Capstone project in Kaggle?")

In [ ]:
# multi agents system
# expense_analyzer
expense_analyzer_agent = Agent(name = "expanse_analyzer_assistant", model = "gemini-2.5-flash-lite",
                              description = "assitant which do analysis of expenses of a person",
                              instruction = "you are a smart anaylsis assistant. You analyze the expenses done by a person in a month.",
                              tools=[google_search],
                              output_key="result")
print("expense_analyzer_agent created.")

In [ ]:
# financial_advisor
financial_advisor_agent = Agent(name = "financial_advisor_assistant", model = "gemini-2.5-flash-lite",
                              description = "assitant who advisor on investment, savings & loan decision to a person",
                              instruction = "you are a smart advisor assistant. On the basis of result of expanse_analyzer_assistant, suggest good financial advise to a person.",
                              tools=[google_search],
                              output_key="advise")
print("financial_advisor_agent created.")

In [ ]:
# root agent - orchestrator
fin_track_agent = SequentialAgent(name = "finTrack", 
                                 sub_agents=[expense_analyzer_agent, financial_advisor_agent])

# Similarly you can create ParallelAgents

print("SequentialAgent fin_track_agent is created.")

In [ ]:
query = "Rahul got salary of 20,000/- today. He has pending bills on his credit cards of around 5,000/- but he decided to buy his favourite video game worth 18,000/-. What should Rahul do?"
print(query)

In [ ]:
runner = InMemoryRunner(agent=fin_track_agent)
response = await runner.run_debug(query)

In [ ]:
# Custom Function Tools -> you own code, solves domain specifi problems, your control over functionality
def risk_assesment(expense_type: str) -> str :
    """Take the expense type and decide the risk invloved with the expense.
    Args: 
        expense_type: String which describes the type of expense like rent, loan, bills, etc.
    Returns:
        String with the resultant risk as evaluated by the internal logic
    """
    if expense_type == "rent":
        return "NECESSITY"
    else:
        return "NON_NECESSITY"

In [ ]:
print(risk_assesment("rent"))

In [ ]:
risk_assesser_agent = LlmAgent(name = "risk_assesment_agent",
                              model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
                              instruction = "You are a smart risk assesment assistant, who identifies the risk from the given expenses of a person",
                              tools = [risk_assesment])
print("risk_assesser_agent is created")

In [ ]:
query = "I have to pay 10,000/- rent from 20,000/- salary. what is the risk level of this expense?"
print(query)

In [ ]:
risk_assesser_runner = InMemoryRunner(agent=risk_assesser_agent)
_ = await risk_assesser_runner.run_debug(query)

In [ ]:
# Agent Tool -> Agent act as a tool, ex. calcAgent which generated python code to do calculations

# Agents vs Sub-agents
# Agents => Delegation for specific tasks (like calculations)
# Sub Agents => Handoff to specialists (like customer support agent)

In [ ]:
# Create a MCP server and call it here
# Idea - MCP for news related to finance & market

In [ ]:
import feedparser

def fetch_yahoo_finance_news(limit=10):
    url = "https://finance.yahoo.com/rss/topstories"
    
    feed = feedparser.parse(url)
    
    articles = []
    for entry in feed.entries[:limit]:
        articles.append({
            "title": entry.title,
            "link": entry.link,
            "published": entry.published
        })
    
    return articles


# ▶️ Fetch and print
articles = fetch_yahoo_finance_news(limit=10)

for idx, a in enumerate(articles, start=1):
    print(f"\n{idx}. {a['title']}")
    print(f"   Link: {a['link']}")
    print(f"   Published: {a['published']}")
